# Objetivo
O objetivo deste trabalho é treinar modelos de classificação binária para prever se um tumor é maligno (M) ou benigno (B), utilizando o Breast Cancer Wisconsin Dataset.
Ao final, o melhor modelo será utilizado para gerar o arquivo submission.csv, no formato:


```
id,label
906564,B
85715,M
...
```

# 1. Exploração de dados

Nesta etapa, foram analisadas as informações básicas do dataset:

Número de instâncias e atributos;

Verificação de valores ausentes;

Distribuição das classes (B e M).

O dataset apresenta 455 registros de treinamento e nenhum valor ausente, o que elimina a necessidade de imputação.
A distribuição das classes mostra 285 casos benignos (B) e 170 malignos (M), revelando um leve desbalanceamento, mas ainda aceitável para treino supervisionado.

In [2]:
import pandas as pd

train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

In [3]:
train.head()

,id,label,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,90524101,M,17.99,20.66,117.80,991.7,0.10360,0.13040,0.120100,0.088240,...,21.080,25.41,138.10,1349.0,0.14820,0.37350,0.33010,0.19740,0.3060,0.08503
1,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.198000,0.104300,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.40000,0.16250,0.2364,0.07678
2,89346,B,9.00,14.40,56.36,246.3,0.07005,0.03116,0.003681,0.003472,...,9.699,20.07,60.90,285.5,0.09861,0.05232,0.01472,0.01389,0.2991,0.07804
3,902975,B,12.21,14.09,78.78,462.0,0.08108,0.07823,0.068390,0.025340,...,13.130,19.29,87.65,529.9,0.10260,0.24310,0.30760,0.09140,0.2677,0.08824
4,904969,B,12.34,14.95,78.29,469.1,0.08682,0.04571,0.021090,0.020540,...,13.180,16.85,84.11,533.1,0.10480,0.06744,0.04921,0.04793,0.2298,0.05974


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 455 entries, 0 to 454
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       455 non-null    int64  
 1   label                    455 non-null    object 
 2   radius_mean              455 non-null    float64
 3   texture_mean             455 non-null    float64
 4   perimeter_mean           455 non-null    float64
 5   area_mean                455 non-null    float64
 6   smoothness_mean          455 non-null    float64
 7   compactness_mean         455 non-null    float64
 8   concavity_mean           455 non-null    float64
 9   concave points_mean      455 non-null    float64
 10  symmetry_mean            455 non-null    float64
 11  fractal_dimension_mean   455 non-null    float64
 12  radius_se                455 non-null    float64
 13  texture_se               455 non-null    float64
 14  perimeter_se             4

In [5]:
train.describe()

,id,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
count,4.550000e+02,455.000000,455.000000,455.000000,455.000000,455.000000,455.000000,455.000000,455.000000,455.000000,...,455.000000,455.000000,455.000000,455.000000,455.000000,455.000000,455.000000,455.000000,455.000000,455.000000
mean,3.494429e+07,14.213492,19.354374,92.572791,664.583077,0.096372,0.105059,0.089651,0.049590,0.181131,...,16.411787,25.705165,108.253319,900.190549,0.132138,0.256131,0.272104,0.115820,0.288476,0.083636
std,1.387826e+08,3.617912,4.399626,24.993837,362.603052,0.013746,0.051977,0.080264,0.039412,0.027257,...,5.013790,6.289274,34.849813,595.178062,0.022190,0.154821,0.204274,0.067030,0.058845,0.016646
min,8.670000e+03,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,...,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156600,0.055210
25%,8.695835e+05,11.705000,16.170000,75.085000,421.950000,0.086730,0.065880,0.028860,0.020335,0.162000,...,12.980000,20.970000,83.680000,511.050000,0.117850,0.149650,0.110900,0.064985,0.250800,0.072090
50%,9.059780e+05,13.400000,18.870000,86.870000,551.100000,0.096390,0.096610,0.063870,0.034830,0.179900,...,14.920000,25.270000,97.660000,684.600000,0.131600,0.218600,0.232200,0.101000,0.281500,0.080090
75%,8.910375e+06,16.090000,21.830000,105.400000,801.550000,0.104900,0.130550,0.132350,0.074975,0.194900,...,19.185000,29.915000,126.900000,1122.500000,0.144800,0.341800,0.385700,0.166100,0.315200,0.091950
max,9.113205e+08,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,...,36.040000,49.540000,251.200000,4254.000000,0.222600,0.937900,1.170000,0.291000,0.577400,0.148600


In [6]:
train['label'].value_counts()

label
B    285
M    170
Name: count, dtype: int64

In [7]:
print(train.isnull().sum())

id                         0
label                      0
radius_mean                0
texture_mean               0
perimeter_mean             0
area_mean                  0
smoothness_mean            0
compactness_mean           0
concavity_mean             0
concave points_mean        0
symmetry_mean              0
fractal_dimension_mean     0
radius_se                  0
texture_se                 0
perimeter_se               0
area_se                    0
smoothness_se              0
compactness_se             0
concavity_se               0
concave points_se          0
symmetry_se                0
fractal_dimension_se       0
radius_worst               0
texture_worst              0
perimeter_worst            0
area_worst                 0
smoothness_worst           0
compactness_worst          0
concavity_worst            0
concave points_worst       0
symmetry_worst             0
fractal_dimension_worst    0
dtype: int64


# 2. Pré-processamento e Engenharia de Atributos

A variável-alvo label foi convertida de texto para valores numéricos:

M → 1

B → 0

Foram removidos os atributos que não são preditivos (id).

Os dados foram padronizados com StandardScaler para normalizar as escalas das variáveis, o que é importante principalmente para modelos baseados em distância (como KNN e SVM).

Essas etapas garantem que todos os atributos contribuam igualmente durante o treinamento.

In [8]:
train['label'] = train['label'].map({'M':1, 'B':0})

In [9]:
from sklearn.preprocessing import StandardScaler

X = train.drop(['label', 'id'], axis=1)
y = train['label']
X_test = test.drop(['id'], axis=1)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)

# 3. Treinamento e Avaliação dos Modelos

Foram testados diferentes algoritmos clássicos de classificação supervisionada:

| Modelo              | Descrição                                | Acurácia (validação) |
| ------------------- | ---------------------------------------- | -------------------- |
| Perceptron          | Classificador linear simples             | 0.9451               |
| Logistic Regression | Modelo linear com função sigmoide        | **1.0000**           |
| SVM                 | Máquina de vetores de suporte            | 0.9890               |
| Decision Tree       | Árvore de decisão                        | 0.9341               |
| KNN                 | Classificação por vizinhos mais próximos | 0.9890               |
| Random Forest       | Conjunto de árvores                      | 0.9451               |


---

**Análise dos resultados:**

A Regressão Logística obteve 100% de acurácia no conjunto de validação, sugerindo que o problema é linearmente separável.

Entretanto, esse desempenho perfeito pode indicar overfitting na divisão específica dos dados.

A SVM também apresentou resultado muito alto (98,9%) e é mais robusta e generalizável.

Modelos baseados em árvores (DecisionTree, RandomForest) tiveram desempenho inferior, possivelmente por capturarem ruído desnecessário.

In [10]:
from sklearn.linear_model import Perceptron, LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, stratify=y, random_state=42)

models = {
    "Perceptron": Perceptron(),
    "LogisticRegression": LogisticRegression(max_iter=1000),
    "SVM": SVC(),
    "DecisionTree": DecisionTreeClassifier(),
    "KNN": KNeighborsClassifier(),
    "RandomForest": RandomForestClassifier()
}

for name, model in models.items():
    model.fit(X_train, y_train)
    pred = model.predict(X_val)
    print(f"{name}: {accuracy_score(y_val, pred):.4f}")


Perceptron: 0.9451
LogisticRegression: 1.0000
SVM: 0.9890
DecisionTree: 0.9560
KNN: 0.9890
RandomForest: 0.9231


# 4. Escolha do Modelo

Diante dos resultados, o modelo SVM (Support Vector Machine) foi escolhido como o mais adequado para a submissão.
Essa decisão se baseia em:


*   Equilíbrio entre desempenho e generalização, já que a SVM tende a ter melhor performance fora da amostra;


*   Estabilidade, mesmo em datasets lineares e com pequeno desbalanceamento;


*   Evita overfitting, diferentemente da regressão logística que obteve 100% no conjunto de validação.

Para refinar o modelo, foi aplicada a técnica de GridSearchCV, buscando os melhores hiperparâmetros (C, kernel, gamma).

O melhor conjunto encontrado foi:

```
{'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'}
```

com acurácia média de 0.9736 em validação cruzada (k=5).


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto']
}

grid = GridSearchCV(SVC(), param_grid, cv=5, scoring='accuracy')
grid.fit(X_scaled, y)

print("Melhores parâmetros:", grid.best_params_)
print("Melhor acurácia média:", grid.best_score_)

Melhores parâmetros: {'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'}
Melhor acurácia média: 0.9736263736263737


# 5. Geração do Arquivo de Submissão

Após o ajuste fino, o modelo final foi utilizado para prever os rótulos (label) do conjunto de teste.
As predições foram convertidas novamente para o formato original (M e B) e inseridas no arquivo sample_submission.csv.

O arquivo final submission.csv contém o identificador (id) e a classificação prevista para cada amostra do conjunto de teste.

Exemplo das primeiras linhas:

| id     | label |
| ------ | ----- |
| 906564 | B     |
| 85715  | M     |
| 891670 | B     |
| 874217 | M     |
| 905680 | M     |


In [12]:
pred_test = grid.best_estimator_.predict(X_test_scaled)
pred_test_labels = ['M' if p==1 else 'B' for p in pred_test]

sample = pd.read_csv("data/sample_submission.csv")
sample['label'] = pred_test_labels
sample.to_csv("submission.csv", index=False)
sample.head()

,id,label
0,906564,B
1,85715,M
2,891670,B
3,874217,M
4,905680,M


# Conclusão
O processo de modelagem demonstrou que o problema de classificação de câncer de mama é altamente linearmente separável, o que explica o excelente desempenho dos modelos lineares.
O modelo final SVM com kernel linear apresentou desempenho médio de 97,36%, equilibrando precisão e generalização, sendo utilizado para a submissão final.